# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy.stats import percentileofscore as score #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2282796638255,
 'week52high': 139.07,
 'week52low': 58.1,
 'week52change': 0.7258299708937976,
 'sharesOutstanding': 16981199308,
 'float': 0,
 'avg10Volume': 111504944,
 'avg30Volume': 116245216,
 'day200MovingAvg': 115.76,
 'day50MovingAvg': 132.98,
 'employees': 0,
 'ttmEPS': 3.32,
 'ttmDividendRate': 0.8201982987334658,
 'dividendYield': 0.006293375426397259,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-24',
 'nextEarningsDate': '0',
 'peRatio': 38.45455823594844,
 'beta': 1.1889528761848205,
 'maxChangePercent': 49.83325492697452,
 'year5ChangePercent': 4.820947326672952,
 'year2ChangePercent': 2.575940022996434,
 'year1ChangePercent': 0.6694710514878103,
 'ytdChangePercent': -0.013967253357017042,
 'month6ChangePercent': 0.38929991326630936,
 'month3ChangePercent': 0.08674417728168213,
 'month1ChangePercent': 0.06966400008103547,
 'day30ChangePercent': 0.07552034766101214,
 'day5ChangePercent': 0.035327027148495574}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.6694710514878103

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['price'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.870,0.475676,N/A
1,AAL,16.710,-0.450028,N/A
2,AAP,172.270,0.197716,N/A
3,AAPL,135.900,0.666301,N/A
4,ABBV,114.980,0.34805,N/A
...,...,...,...,...
500,YUM,111.309,0.0898693,N/A
501,ZBH,166.190,0.102281,N/A
502,ZBRA,427.750,0.689814,N/A
503,ZION,51.600,0.0131695,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.900,2.48136,N/A
1,ALB,183.880,1.43349,N/A
2,LB,49.320,1.3405,N/A
3,FCX,32.180,1.30943,N/A
4,NVDA,556.870,1.20586,N/A
5,PYPL,243.945,1.12627,N/A
6,ABMD,321.850,0.943686,N/A
7,WST,304.190,0.928104,N/A
8,AMD,94.513,0.904686,N/A
9,PWR,79.067,0.855278,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe

C:\Users\PLT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.900,2.48136,4773
1,ALB,183.880,1.43349,1087
2,LB,49.320,1.3405,4055
3,FCX,32.180,1.30943,6215
4,NVDA,556.870,1.20586,359
5,PYPL,243.945,1.12627,819
6,ABMD,321.850,0.943686,621
7,WST,304.190,0.928104,657
8,AMD,94.513,0.904686,2116
9,PWR,79.067,0.855278,2529


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.660,N/A,0.485961,N/A,0.439606,N/A,0.212024,N/A,0.0741777,N/A,N/A
1,AAL,16.381,N/A,-0.438433,N/A,0.341444,N/A,0.277376,N/A,-0.0918891,N/A,N/A
2,AAP,175.740,N/A,0.199354,N/A,0.279678,N/A,0.0953284,N/A,0.0945917,N/A,N/A
3,AAPL,131.938,N/A,0.671557,N/A,0.389378,N/A,0.0862081,N/A,0.0696211,N/A,N/A
4,ABBV,116.494,N/A,0.344822,N/A,0.183476,N/A,0.298049,N/A,0.0592973,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.661,N/A,0.092402,N/A,0.242547,N/A,0.122273,N/A,0.0114895,N/A,N/A
501,ZBH,168.190,N/A,0.102575,N/A,0.367249,N/A,0.131402,N/A,0.155796,N/A,N/A
502,ZBRA,429.980,N/A,0.677887,N/A,0.626742,N/A,0.452007,N/A,0.0932989,N/A,N/A
503,ZION,51.000,N/A,0.0133662,N/A,0.548018,N/A,0.650511,N/A,0.195841,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [22]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0
        else:
            hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
            
#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.660,N/A,0.485961,89.3069,0.439606,71.2871,0.212024,63.9604,0.0741777,65.9406,N/A
1,AAL,16.381,N/A,-0.438433,1.38614,0.341444,58.6139,0.277376,72.6733,-0.0918891,1.38614,N/A
2,AAP,175.740,N/A,0.199354,63.7624,0.279678,50.099,0.0953284,41.7822,0.0945917,74.4554,N/A
3,AAPL,131.938,N/A,0.671557,95.0495,0.389378,65.3465,0.0862081,40.198,0.0696211,63.5644,N/A
4,ABBV,116.494,N/A,0.344822,80,0.183476,35.4455,0.298049,76.0396,0.0592973,58.4158,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.661,N/A,0.092402,48.7129,0.242547,45.3465,0.122273,48.7129,0.0114895,32.4752,N/A
501,ZBH,168.190,N/A,0.102575,50.6931,0.367249,61.3861,0.131402,49.703,0.155796,90.099,N/A
502,ZBRA,429.980,N/A,0.677887,95.2475,0.626742,88.1188,0.452007,88.1188,0.0932989,73.6634,N/A
503,ZION,51.000,N/A,0.0133662,35.6436,0.548018,84.7525,0.650511,96.0396,0.195841,95.6436,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [23]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.660,N/A,0.485961,89.3069,0.439606,71.2871,0.212024,63.9604,0.0741777,65.9406,72.6238
1,AAL,16.381,N/A,-0.438433,1.38614,0.341444,58.6139,0.277376,72.6733,-0.0918891,1.38614,33.5149
2,AAP,175.740,N/A,0.199354,63.7624,0.279678,50.099,0.0953284,41.7822,0.0945917,74.4554,57.5248
3,AAPL,131.938,N/A,0.671557,95.0495,0.389378,65.3465,0.0862081,40.198,0.0696211,63.5644,66.0396
4,ABBV,116.494,N/A,0.344822,80,0.183476,35.4455,0.298049,76.0396,0.0592973,58.4158,62.4752
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.661,N/A,0.092402,48.7129,0.242547,45.3465,0.122273,48.7129,0.0114895,32.4752,43.8119
501,ZBH,168.190,N/A,0.102575,50.6931,0.367249,61.3861,0.131402,49.703,0.155796,90.099,62.9703
502,ZBRA,429.980,N/A,0.677887,95.2475,0.626742,88.1188,0.452007,88.1188,0.0932989,73.6634,86.2871
503,ZION,51.000,N/A,0.0133662,35.6436,0.548018,84.7525,0.650511,96.0396,0.195841,95.6436,78.0198


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [24]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:50]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [25]:
portfolio_input()

Enter the value of your portfolio:10000000


In [26]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\PLT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.660,1519,0.485961,89.3069,0.439606,71.2871,0.212024,63.9604,0.0741777,65.9406,72.6238
1,AAL,16.381,12209,-0.438433,1.38614,0.341444,58.6139,0.277376,72.6733,-0.0918891,1.38614,33.5149
2,AAP,175.740,1138,0.199354,63.7624,0.279678,50.099,0.0953284,41.7822,0.0945917,74.4554,57.5248
3,AAPL,131.938,1515,0.671557,95.0495,0.389378,65.3465,0.0862081,40.198,0.0696211,63.5644,66.0396
4,ABBV,116.494,1716,0.344822,80,0.183476,35.4455,0.298049,76.0396,0.0592973,58.4158,62.4752
5,ABC,108.770,1838,0.248431,70.495,0.0985248,21.3861,0.0943232,41.5842,0.0807994,69.3069,50.6931
6,ABMD,333.000,600,0.927043,98.6139,0.234348,43.5644,0.170536,56.4356,0.261361,98.4158,74.2574
7,ABT,114.340,1749,0.33841,79.802,0.219281,41.5842,0.0393847,30.6931,0.0481227,53.4653,51.3861
8,ACN,256.410,780,0.254119,70.8911,0.21499,40.5941,0.129421,49.1089,0.0511832,55.0495,53.9109
9,ADBE,476.790,419,0.378454,83.3663,0.0674252,17.8218,-0.0841953,7.32673,-0.0083567,21.9802,32.6238


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [27]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [29]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [30]:
writer.save()